<a href="https://colab.research.google.com/github/jpoist97/lab-6-jpoist97/blob/part-b/DATA_301_Lab_6B_Justin_Poist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bing Maps API

In this part of the lab, you will join the CityBikes data from the previous part of the lab with additional data that you will query from the Bing Maps API.

First, you will need to register for a Bing Maps Key. Follow the instructions [here](https://docs.microsoft.com/en-us/bingmaps/getting-started/bing-maps-dev-center-help/getting-a-bing-maps-key). You should be able to sign in with your Cal Poly account. When you get to Step 4, select:

- Key type: Basic
- Application type: Dev/Test

You will be able to make 125000 free requests with the resulting API key. This should be more than enough to complete this assignment.

We will be working with the [REST services in the Bing Maps API](https://docs.microsoft.com/en-us/bingmaps/rest-services/). Click on the link for a complete documentation of the features.

## Question 1

Read in the `DataFrame` of bike stations in the United States from Part A of this lab. Restrict to the stations in the "Bay Wheels" network (with network ID "ford-gobike").

How many of these stations are in the city/county of San Francisco? 

(_Hint:_ Use the [Locations API](https://docs.microsoft.com/en-us/bingmaps/rest-services/locations/) to get the address associated with each latitude and longitude coordinate.)

In [66]:
import pandas as pd
import requests
import time

df_bike_stations = pd.read_csv("bike_stations.csv")

df_bay_wheels = df_bike_stations[df_bike_stations["network_id"] == "bay-wheels"]

num_sf = 0
df_sf_bike_share = pd.DataFrame(columns=df_bay_wheels.columns.tolist())



def find_county(row_series):
  latitude = row_series["latitude"]
  longitude = row_series["longitude"]

  response = requests.get("http://dev.virtualearth.net/REST/v1/Locations/" + str(latitude) + "," + str(longitude) + "?key=AgEzxjqI_hrzgJtK3P0q4dlzWf8aILYsp_uW3OEp9miAOUr7eNyXVFILjBBJJz2_")

  county = response.json()["resourceSets"][0]["resources"][0]["address"]["adminDistrict2"]
  time.sleep(0.3)
  return county


df_bay_wheels["county"] = df_bay_wheels.apply(find_county, axis=1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,network_id,extra.address,extra.last_updated,extra.renting,extra.returning,extra.uid,extra.ebikes,extra.has_ebikes,extra.bike_uids,extra.number,extra.slots,county
6231,11.0,4,d0e8f4f1834b7b33a3faf8882f567ab8,37.849735,-122.270582,Harmon St at Adeline St,2022-02-22T22:24:04.781000Z,bay-wheels,NaN,1.645564e+09,1.0,1.0,340,0.0,True,NaN,NaN,NaN,Alameda Co.
6232,5.0,6,983514094dd808b1604da2dcfc2d09af,37.336188,-121.889277,Fountain Alley at S 2nd St,2022-02-22T22:24:04.790000Z,bay-wheels,NaN,1.645533e+09,1.0,1.0,341,0.0,True,NaN,NaN,NaN,Santa Clara Co.
6233,2.0,21,da17603652106fda93da4e255a5b0a22,37.322125,-121.881090,Oak St at 1st St,2022-02-22T22:24:04.816000Z,bay-wheels,NaN,1.645554e+09,1.0,1.0,298,1.0,True,NaN,NaN,NaN,Santa Clara Co.
6234,2.0,13,7a21c92b3b4cd2f7759107b4fdebf869,37.323678,-121.874119,Bestor Art Park,2022-02-22T22:24:04.821000Z,bay-wheels,NaN,1.645532e+09,1.0,1.0,299,0.0,True,NaN,NaN,NaN,Santa Clara Co.
6235,21.0,6,ce34d38fb230a23c1ced12d1e16df294,37.325998,-121.877120,5th St at Virginia St,2022-02-22T22:24:04.822000Z,bay-wheels,NaN,1.645550e+09,1.0,1.0,296,1.0,True,NaN,NaN,NaN,Santa Clara Co.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,17.0,6,df9c9c79952699cc041118a331ea9cc2,37.724893,-122.443238,Balboa Park (San Jose Ave at Sgt. John V. Youn...,2022-02-22T22:24:06.111000Z,bay-wheels,NaN,1.645535e+09,1.0,1.0,501,5.0,True,NaN,NaN,NaN,San Francisco Co.
6709,16.0,11,5dc0d98ea835b4c9743d4e2cdd515dde,37.726266,-122.479462,SFSU University Park North - West,2022-02-22T22:24:06.354000Z,bay-wheels,NaN,1.645552e+09,1.0,1.0,566,0.0,True,NaN,NaN,NaN,San Francisco Co.
6710,3.0,12,97871d03b345e3aa1de1dbed3640b700,37.725603,-122.476561,SFSU University Park North - East,2022-02-22T22:24:06.357000Z,bay-wheels,NaN,1.645552e+09,1.0,1.0,567,8.0,True,NaN,NaN,NaN,San Francisco Co.
6711,4.0,15,e427fa5d07c952f21297dbd9c7693407,37.723426,-122.455102,Brighton Ave at Ocean Ave,2022-02-22T22:24:06.352000Z,bay-wheels,NaN,1.645535e+09,1.0,1.0,565,5.0,True,NaN,NaN,NaN,San Francisco Co.


In [69]:
df_sf_bike_share = df_bay_wheels[df_bay_wheels["county"] == "San Francisco Co."]
df_sf_bike_share.shape[0]

271

There are 271 bike stations in the county of San Francisco.

## Question 2

You want to go to Coit Tower. To save money, you decide to ride a "Bay Wheels" bike to the closest station and hail a cab from there to Coit Tower. What station should you bike to so that you are as close to Coit Tower as possible (as measured by driving distance)? Does your answer agree with the one that you obtained in Part A of this lab? If not, why does it differ?

_Hints:_ 
- You should restrict your attention to bike stations that are in San Francisco, which you determined in Question 1. 
- Use the [Routes API](https://docs.microsoft.com/en-us/bingmaps/rest-services/routes/calculate-a-distance-matrix) to calculate a distance matrix between Coit Tower and the bike stations. 
- You can do this with just one call to the API. Because there are too many stations, it is impossible to specify all the locations in the URL. Instead, you should make a POST request (`requests.post`), passing in the parameters as a JSON object through the `json=` parameter of `requests.post`. Read the API documentation carefully to learn how to use the POST API.

In [90]:

def build_request_body():
  request_body = {
      "origins": [],
      "destinations": [{
        "latitude": 37.8024,
        "longitude": -122.4058
      }],
      "travelMode": "Driving"
  }
  for row_series in df_sf_bike_share.iterrows():
    request_body["origins"].append({
        "latitude": row_series[1]["latitude"],
        "longitude": row_series[1]["longitude"]
    })

  return request_body

response = requests.post("https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?key=AgEzxjqI_hrzgJtK3P0q4dlzWf8aILYsp_uW3OEp9miAOUr7eNyXVFILjBBJJz2_", json=build_request_body())


In [95]:
df_distance_bike_share = pd.json_normalize(response.json()["resourceSets"][0]["resources"][0]["results"])
df_distance_bike_share = df_distance_bike_share.join(df_sf_bike_share.reset_index())

min_index = df_distance_bike_share["travelDuration"].idxmin()
df_distance_bike_share.loc[min_index]

destinationIndex                                     0
originIndex                                        218
totalWalkDuration                                    0
travelDistance                                   0.965
travelDuration                                  3.2333
index                                             6647
empty_slots                                        8.0
free_bikes                                          11
id                    65134e1e5e8d3b200c458d533e7d544c
latitude                                     37.802767
longitude                                  -122.413276
name                        Lombard St at Columbus Ave
timestamp                  2022-02-22T22:24:05.309000Z
network_id                                  bay-wheels
extra.address                                      NaN
extra.last_updated                        1645568523.0
extra.renting                                      1.0
extra.returning                                    1.0
extra.uid 

The station that I'd bike to based on this exercise is the station at latitude 37.802767 and longitude -122.413276. This is different than the answer in the previous lab part which was the station at latitude 37.714145 and longitude -122.25.

I'm not too sure why the two answers would differ, but if I had to guess it would be because the manhattan distance we used in the previous lab part doesn't actually account for how the streets of San Francisco are set up. Instead it uses teh mathematical manhattan distance between two numbers, which isn't the most useful in this scenario.

## Submission Instructions

- Copy this notebook to your own Drive, if you have not already.
- Restart this notebook and run the cells from beginning to end. 
  - Go to Runtime > Restart and Run All.
- Rename this notebook by clicking on "DATA 301 Lab 6B - YOUR NAMES HERE" at the very top of this page. Replace "YOUR NAMES HERE" with the first and last names of you (and your partners, for Phase 2).
- Get the link to your notebook:
  - Click on "Share" at the top-right. 
  - Change the settings to "Anyone with the link can view". 
  - Copy the sharing link into Canvas.